In [1]:
from itertools import chain
import json

meta = {}
with open('../datasets/tracktranslegislation-meta.json', 'r') as f:
    meta = json.load(f)

aggregated = {}
with open('../artifacts/aggregate.json', 'r') as f:
    aggregated = json.load(f)

def categorize_aggregate(aggregate, keywords):
    return set(
        keyword[0]
        for keyword
        in keywords
        if (
            keyword[1] in aggregate['title'].lower()
            or
            keyword[1] in aggregate['description'].lower()
        )
    )

keywords = list(chain.from_iterable(
    ((category['name'], keyword) for keyword in category['keywords'])
    for category
    in meta
))

categorized = list(
    (
        aggregate['state'], 
        aggregate['bill_id'], 
        categorize_aggregate(aggregate, keywords)
    )
    
    for aggregate
    in aggregated
)

grouped = {
    category['name']: [
        categorize[0:2]
        for categorize
        in categorized
        if category['name'] in categorize[2]
    ]
    for category
    in meta
}

grouped['Uncategorized'] = [
    categorize[0:2]
    for categorize
    in categorized
    if len(categorize[2]) == 0
]

with open('../artifacts/categorized.json', 'w') as f:
    json.dump(grouped, f, indent=2)